Import Libraries

In [1]:
from pyvis.network import Network

Define initial parameters

In [2]:
# define time steps
timeSteps = 100

Create Model

In [3]:
net = Network()
net.add_node(0, label="initial scholar")

# go through time steps
for i in range(1, timeSteps):
    net.add_node(i)

Display Network

In [7]:
net.show('testModel.html')